In [1]:
import requests
import yaml
import json
import gzip
import pandas as pd

In [4]:
mecab_tokenize("あああ")

ValueError: too many values to unpack (expected 2)

In [2]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

/home/yamada/.local/share/virtualenvs/twichAI-ydQv36PI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# payload = {'out': 'json', 'lim':2}
payload = {'out': 'json','gzip':1,'of':'n','lim':50}
res = requests.get('https://api.syosetu.com/novelapi/api/', params=payload).content
r =  gzip.decompress(res).decode("utf-8") 
allcount = json.loads(r)[0]["allcount"]
print(r)

[{"allcount":881841},{"ncode":"N3218HM"},{"ncode":"N2129HM"},{"ncode":"N9097HP"},{"ncode":"N2640HN"},{"ncode":"N2613HO"},{"ncode":"N3168FR"},{"ncode":"N0155HN"},{"ncode":"N6693HP"},{"ncode":"N0116HQ"},{"ncode":"N7883HK"},{"ncode":"N4090GW"},{"ncode":"N8171HM"},{"ncode":"N9978HN"},{"ncode":"N1120GU"},{"ncode":"N6246HP"},{"ncode":"N2840HD"},{"ncode":"N0242HQ"},{"ncode":"N0423HM"},{"ncode":"N4764HN"},{"ncode":"N6242HP"},{"ncode":"N9265HP"},{"ncode":"N8849HM"},{"ncode":"N0241HQ"},{"ncode":"N7791HO"},{"ncode":"N2958HP"},{"ncode":"N9696HJ"},{"ncode":"N7459HP"},{"ncode":"N0175HQ"},{"ncode":"N2652HN"},{"ncode":"N7934GS"},{"ncode":"N9016HO"},{"ncode":"N4231HP"},{"ncode":"N1463HI"},{"ncode":"N0721GL"},{"ncode":"N9587HP"},{"ncode":"N5543HO"},{"ncode":"N2857HO"},{"ncode":"N7901HJ"},{"ncode":"N0866HM"},{"ncode":"N6148HC"},{"ncode":"N6630HK"},{"ncode":"N0248HN"},{"ncode":"N9296HP"},{"ncode":"N5667HN"},{"ncode":"N6943HP"},{"ncode":"N8548HM"},{"ncode":"N9990HN"},{"ncode":"N8888HP"},{"ncode":"N8542HP"}

In [4]:
allcount

881841

In [5]:
payload = {'out': 'json','gzip':5,'order':'weeklypoint','lim':500}
res = requests.get("https://api.syosetu.com/novelapi/api/?genre=102&length=1000-&type=r", params=payload, timeout=30).content
r =  gzip.decompress(res).decode("utf-8")
df_temp = pd.read_json(r)

In [6]:
df_temp
df_temp = df_temp.drop(0)

In [7]:
ncodes = list(df_temp.ncode.values)

In [15]:
len(ncodes)

500

In [8]:
from urllib import request
from bs4 import BeautifulSoup
import re


In [9]:
def get_one_page(url):
    # url = "http://www.yomiuri.co.jp/"
    text_list = []
    try:
        html = request.urlopen(url)

        #set BueatifulSoup
        soup = BeautifulSoup(html, "html.parser")

        #get 本文
        contents = soup.find("div", id="novel_honbun", class_="novel_view")
        contents_p = contents.find_all("p")
        
        for text in contents_p:
            # line = re.sub("<.+>", "", str(line))
            text = text.get_text().strip()
            if not text :
                continue
            text_list.append(text)

        return text_list
        
    except Exception as E:
        return text_list
    

In [18]:
base_url = "https://ncode.syosetu.com/"
def crawl_one_novel(ncode:str, start=1, end=10):
    novel_url = base_url+ncode.lower()+"/"
    novel = []
    for i in range(start, end+1) :
        url = novel_url+str(i)
        page = get_one_page(url)
        if page==[]:
            break
        novel += page
    return novel

In [11]:
def clean_text_plain(text):
    text_ = neologdn.normalize(text)
    text_ = re.sub(r'\(.*\)', "", text_)
    text_ = re.sub(r'\d+', "0", text_)
    if "……" in text_:
        text_ = text_.replace("……", "…")
    return text_

In [12]:
def format_EOS(sentence):
    if sentence[-1] in ["。", "?", "!", "．"]:
        return sentence
    else:
         return sentence+"。"

In [13]:
def is_valid_utterance_segment(text:str)->bool:
    # 複数の"「"には囲まれていない
    if text.count("「") == 1 and text.count("」") == 1:
        return True
    return False

In [16]:
pattern = '(?<=\「).*(?=\」)'
for ncode in ncodes:
    print(ncode)
    novel = crawl_one_novel(ncode, end=50)
    convs = []
    for text in tqdm(novel):
        # if is_valid_utterance_segment(text):
        #     utt = re.search(pattern, text).group()
        #     utt = clean_text_plain(utt)
        #     utt = format_EOS( utt )
        #     convs.append(utt)
        print(text)
    # print(convs)
    break

  0%|          | 0/10 [00:00<?, ?it/s]

N0855HP


100%|██████████| 918/918 [00:00<00:00, 9233.70it/s]

「ねえねえお兄ちゃん」
「お兄ちゃんって呼ぶな」
自分にちっとも似ていない妹に向かって、何度目かわからない注意をする。
同じ苗字であり、一つ屋根の下で暮らしているのだから、関係上はそう呼ばれても問題ない。
だが、彼女――義理の妹である悠羽（ゆうは）に、兄として扱われることが嫌だった。
「じゃあ、六郎（ろくろう）」
「それでいい」
不満げに言う少女に頷くと、ますます彼女は顔を曇らせる。
「どうして六郎は、お兄ちゃんって呼ばれたくないの」
「それは――」
いつもここで、話を変える。お茶を濁す。でっち上げた適当な言い訳でやり過ごす。
――俺は悠羽の兄ではない。
だが、彼女はそれを知らない。
そんなことは、知らなくてもいいと思っていた。
◇
「紹介するぜ。この子が俺の彼女――荒川（あらかわ）奈子（なこ）ちゃんです！」
「よろしくお願いしまぁす」
学生街にある居酒屋の個室で、向かいに座った男女一組。
男の方は高校から付き合いのある新田（にった）圭次（けいじ）。
女の方は初対面で、きれいに化粧をした華のある子だ。
相対する俺は二人分の席を一人で占有し、目の前にある不愉快な現実の中、必死に自我を保っていた。
「よく来てくれたなサブ。今日は俺の奢りでいいから、思いっきり呑んで話そうぜ」
圭次は大層嬉しそうにニコニコ笑って、メニューを渡してくる。飯を奢ると言うから来てみれば、彼女自慢が本題らしい。
「奢りじゃ足りねえよバカ。賠償金寄越せタコ」
「馬刺しと揚げタコな。酒は梅酒でいいんだっけ？　梅酒（ばいしゅ）金つってな」
「無敵かよお前。つーかここ、馬刺しなんて売ってないだろ」
こっちの罵倒をことごとくつまらないギャグにして、圭次はへらへらしている。
その横で女は、口元を押さえて笑っていた。きっとあれは笑っているフリだろう、と思うことにする。
「おうよ。俺様は今、天下無敵の気分だぜ。いや～、自分を想ってくれる人がいるってのは、いいもんだなぁ」
「うふふ。圭次さんってば、もう酔ってるんですか？」
「来る前に一杯飲んだだーけ」
「もうっ。圭次さんったら」
キャッキャうふふと腕を絡め合う二人。
……あの、俺さっさと帰っていいですか？
できたてカップルの正面で食う飯が世界で一番不味い。それがずっと付き合いのある友人であれば、なおさらだ。
人の幸福は泥の味とは前々から思っていたが、友人となると格

In [40]:
novel

['「ねぇねぇオタク君」',
 'どこにでも居る、金髪ギャルの鳴海（なるみ）\u3000優愛（ゆあ）高校１年生。',
 '彼女がクラスメイトである小田倉（おたくら）\u3000浩一（こういち）に話しかけたのは、ただの気まぐれだった。',
 '放課後にいつも話す友達が居なかったので、暇つぶしにと、近くの席に居た小田倉に声をかけただけだった。',
 '「えっ、オタク君って僕の事？」',
 '「そうだよ。なんで？」',
 '小田倉は返答に困った。',
 '彼は確かにオタクではある。',
 'だが、かつて中学時代はそれで馬鹿にされたこともあり、高校になってからは極力周りにバレないようにしてきたつもりだ。',
 'だと言うのに、いきなりクラスメイトのギャルにオタク君呼ばわりされたのだ。',
 '（オタク呼ばわりは酷いけど、オタク君って呼ばれるのは嬉しいかも）',
 '彼はちょっとだけ、喜んだ。',
 '見事にツ〇ッター漫画に毒されているオタクである。',
 '「苗字が小田倉なだけで、オタクじゃないよ」',
 '「でもオタク君、授業中に女の子の絵描いてたりするじゃん？」',
 '「なっ！？」',
 '小田倉ことオタク君は気づいていなかった。',
 '自分の前や横の席から見えないようにしていても、自分の斜め後ろの席からは丸見えだったという事が。',
 '幸いにして彼の席は窓際の後ろから二番目。気づいて居るのは彼女だけだ。',
 '「そ、それで用事は何かな？」',
 '本人はさり気ないつもりだが、必死に話題を変えているのは一目瞭然だ。',
 '「それがさ、このネイル。ヤバくね？」',
 '優愛は彼の動揺など気にもせず、そう言って、手を出した。',
 '彼女にとって、オタク君が絵を描いている事など、どうでも良いからだ。',
 'オタク君がオタク趣味でも別に良くない？',
 'その程度の認識だ。',
 '全く興味のない爪を見せられた上に「ヤバくね？」と言われ、言葉を失うオタク君。',
 '人の事を「オタク君」と呼ぶ事の方がヤバくねと言いたい気持ちを、ぐっとこらえる。',
 '「えっと、どうヤバいのかな？」',
 '「これ自分で塗ってみたんだけど、どうよ？」',
 '優愛の爪を見るオタク君。',
 '爪にはカラフル、なんて形容詞では誤魔化せないような模様が出来ていた。',
 '絵画の授業

In [269]:
class Collector:
    def __init__(self, out_path="../../corpus/novel/"):
        self.out_path = out_path
        self.utt_set = set()
        self.title_set = set()
    
    def crawl(self, limit=5000):
        # limit 以下の場合に実行
        
        # 小説タイトルが沢山
        ncodes = self.get_titles()
        for i, ncode in enumerate(ncodes):
            # print(ncode)
            # タイトルが既に登録されていれば無視
            if ncode in self.title_set:
                continue
            # 会話文のみ
            convs = self.extract_convs_from_ncode(ncode, end=50)
            self.utt_set.update(set(convs))
                # 追加したタイトルを保存
            self.title_set.add(ncode)

            # 進捗
            if (i+1) % 10 == 0:
                print("complete rate => {0} : {1}%".format(len(self.utt_set), len(self.utt_set)*100//limit))
            
            # 上限を超えた！
            if len(self.utt_set) >= limit:
                break
        
        print("ended crawl/ crawled data => {0} : {1}%".format(len(self.utt_set), len(self.utt_set)*100//limit))
            
    
    def extract_convs_from_ncode(self, ncode, end=100) -> list:
        pattern = '(?<=\「).*(?=\」)'
        novel = crawl_one_novel(ncode, end=end)
        convs = []
        for text in novel:
            if "「" in text and "」" in text:  
                # print(text)
                utt = re.search(pattern, text).group()
                utt = clean_text_plain(utt)
                if utt == "":
                    continue
                # print(utt)
                utt = format_EOS( utt ) 
                convs.append(utt)
        return convs
    
    def get_titles(self):
        payload = {'out': 'json','gzip':5,'order':'yearlypoint','lim':500}
        res = requests.get("https://api.syosetu.com/novelapi/api/?genre=102&length=100000-&type=r", params=payload, timeout=30).content
        r =  gzip.decompress(res).decode("utf-8")
        df_temp = pd.read_json(r)
        df_temp = df_temp.drop(0)
        ncodes = list(df_temp.ncode.values)
        return ncodes
    
    def save(self, filename, titles):
        conv_data = dict()
        conv_data["utt"] = sorted( list(self.utt_set) ) 

        title_data = dict()
        title_data["ncode"] = sorted( list(self.title_set) ) 

        with open(self.out_path+filename, mode="w") as f:
            json.dump(conv_data, f, ensure_ascii=False, indent=4)
        
        with open(self.out_path+titles, mode="w") as f:
            json.dump(title_data, f, ensure_ascii=False, indent=4)
        
    def load(self, filename, titles):
        # 実データ
        if not os.path.exists(self.out_path+filename):
            print("file was not found : {0}".format(self.out_path+filename))
            return False
        else:
            # jsonファイル
            with open(self.out_path+filename, mode="r") as f:
                data = json.load(f)
            self.utt_set = set(data["utt"])
            print("success load : {0}".format(self.out_path+filename))
        
        # 読み込んだタイトル達
        if not os.path.exists(self.out_path+titles):
            print("file was not found : {0}".format(self.out_path+titles))
            return False

        else:
            with open(self.out_path+titles, mode="r") as f:
                data = json.load(f)
            self.ncode_set = set(data["ncode"])
            print("success load : {0}".format(self.out_path+titles))
        
        return True

In [270]:
collector = Collector()

In [271]:
filename = "novels.json"
titles = "ncodes.json"

In [272]:
collector.load(filename, titles)

file was not found : ../../corpus/novel/novels.json


False

In [43]:
base_url = "https://ncode.syosetu.com/"
def crawl_one_novel_div_page(ncode:str, start=1, end=10):
    novel_url = base_url+ncode.lower()+"/"
    novel = []
    for i in range(start, end+1) :
        url = novel_url+str(i)
        page = get_one_page(url)
        if page==[]:
            break
        novel.append(page)
    return novel

In [94]:
class Collector2:
    def __init__(self, out_path="../../corpus/novel2/"):
        self.out_path = out_path
        self.utt_set = set()
        self.title_set = set()
    
    def crawl(self, limit=5000, num_page=100):
        # limit 以下の場合に実行
        
        # 小説タイトルが沢山
        ncodes = self.get_titles(limit)
        for i, ncode in enumerate(ncodes):
            
            # タイトルが既に登録されていれば無視
            if ncode in self.title_set:
                continue
            # print(ncode)
            # 会話文のみ
            contents = self.extract_contents_from_ncode(ncode, end=num_page)
            # self.utt_set.update(set(convs))
            # 小説の保存
            self.write(ncode, contents)
                # 追加したタイトルを保存
            self.title_set.add(ncode)

            # 進捗
            if (i+1) % 20 == 0:
                print("complete rate => {0} : {1}%".format(i+1, (i+1)*100//len(ncodes)))
        
        print("ended crawl/ crawled data => {0} : {1}%".format(i+1, (i+1)*100//len(ncodes)))
            
    
    def extract_contents_from_ncode(self, ncode, end=100) -> list:
        novel = crawl_one_novel_div_page(ncode, end=end)
        contents = []
        # print(novel)
        # for text in novel:
            # if "「" in text and "」" in text:  
            #     # print(text)
            #     utt = re.search(pattern, text).group()
            #     utt = clean_text_plain(utt)
            #     if utt == "":
            #         continue
            #     # print(utt)
            #     utt = format_EOS( utt ) 
                # contents.append(text)
        return novel
    
    def get_titles(self, limit):
        payload = {'out': 'json','gzip':5,'order':'yearlypoint','lim':limit}
        res = requests.get("https://api.syosetu.com/novelapi/api/?genre=102&length=100000-&type=r", params=payload, timeout=30).content
        r =  gzip.decompress(res).decode("utf-8")
        df_temp = pd.read_json(r)
        df_temp = df_temp.drop(0)
        ncodes = list(df_temp.ncode.values)
        return ncodes
    
    def save(self, filename, titles):
        # conv_data = dict()
        # conv_data["utt"] = sorted( list(self.utt_set) ) 

        title_data = dict()
        title_data["ncode"] = sorted( list(self.title_set) ) 

        # with open(self.out_path+filename, mode="w") as f:
        #     json.dump(conv_data, f, ensure_ascii=False, indent=4)
        
        with open(self.out_path+titles, mode="w") as f:
            json.dump(title_data, f, ensure_ascii=False, indent=4)
        
    def load(self, filename, titles):
        # 実データ
        # if not os.path.exists(self.out_path+filename):
        #     print("file was not found : {0}".format(self.out_path+filename))
        #     return False
        # else:
        #     # jsonファイル
        #     with open(self.out_path+filename, mode="r") as f:
        #         data = json.load(f)
        #     self.utt_set = set(data["utt"])
        #     print("success load : {0}".format(self.out_path+filename))
        
        # 読み込んだタイトル達
        if not os.path.exists(self.out_path+titles):
            print("file was not found : {0}".format(self.out_path+titles))
            return False

        else:
            with open(self.out_path+titles, mode="r") as f:
                data = json.load(f)
            self.title_set = set(data["ncode"])
            print("success load : {0}".format(self.out_path+titles))
            self.title_set.update(set(os.listdir(self.out_path)))
        
        return True

    def write(self, ncode, contents):
        os.makedirs(self.out_path+ncode, exist_ok=True)
        for i, content in enumerate(contents):
            # print(content)
            with open(self.out_path+ncode+"/"+"{0}.txt".format(i+1), "w") as f:
                f.write("\n".join(content))

In [95]:
filename = "novels_{0}.json".format(50)
titles = "ncodes2_{0}.json".format(50)

In [96]:
# collector.save(filename, titles)

In [97]:
col = Collector2()
col.load("", titles)

success load : ../../corpus/novel2/ncodes2_50.json


True

In [98]:
len(col.title_set)

229

In [99]:
col.crawl(limit=500, num_page=100)

complete rate => 240 : 48%
complete rate => 260 : 52%
complete rate => 280 : 56%
complete rate => 300 : 60%
complete rate => 320 : 64%
complete rate => 340 : 68%
complete rate => 360 : 72%
complete rate => 380 : 76%
complete rate => 400 : 80%
complete rate => 420 : 84%
complete rate => 440 : 88%
complete rate => 460 : 92%
complete rate => 480 : 96%
complete rate => 500 : 100%
ended crawl/ crawled data => 500 : 100%


In [100]:
col.save("", titles)